In [3]:
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import plotly.plotly as py
from plotly import tools
from datetime import date
import pandas as pd
import numpy as np 
import plotly.figure_factory as ff

from datetime import date
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

import matplotlib.pyplot as plt
import seaborn as sns
import random 
import warnings
import operator
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

In [2]:
#need to figure out a way to manage missing values

## Exploration of the data

In [180]:

train = pd.read_csv("../Costa_rica_poverty_prediction/costa-rican-household-poverty-prediction/train.csv", nrows = None)
test = pd.read_csv("../Costa_rica_poverty_prediction/costa-rican-household-poverty-prediction/test.csv")


#how to set wdir ??
print ("Train Dataset: Rows, Columns: ", train.shape)
print ("Test Dataset: Rows, Columns: ", test.shape)
#weird that we have bigger test set than train set

Train Dataset: Rows, Columns:  (9557, 143)
Test Dataset: Rows, Columns:  (23856, 142)


## Feature engineering

Code written by me

In [156]:
def feature_creation_function(df):
    feats_div = [('children_fraction', 'r4t1', 'r4t3'), 
                 ('working_man_fraction', 'r4h2', 'r4t3'),
                 ('all_man_fraction', 'r4h3', 'r4t3'),
                 ('human_density', 'tamviv', 'rooms'),
                 ('human_bed_density', 'tamviv', 'bedrooms'),
                 ('bed_density', 'bedrooms', 'rooms'),
                 ('rent_per_person', 'v2a1', 'r4t3'),
                 ('rent_per_room', 'v2a1', 'rooms'),
                 ('person_per_room', 'r4t3', 'rooms'),
                 ('person_per_room', 'r4t3', 'bedrooms'),
                 ('rez_esc_escolari', 'rez_esc', 'escolari'),
                 ('rez_esc_r4t1', 'rez_esc', 'r4t1'),
                 ('mobile_density', 'qmobilephone', 'r4t3'),
                 ('tablet_density', 'v18q1', 'r4t3'),
                 ('mobile_adult_density', 'qmobilephone', 'r4t2'),
                 ('tablet_adult_density', 'v18q1', 'r4t2'),
                 ('male_over_female', 'r4h3', 'r4m3'),
                 ('man12plus_over_women12plus', 'r4h2', 'r4m2'),
                 ('pesioner_over_working', 'hogar_mayor', 'hogar_adul'),
                 ('children_over_working', 'hogar_nin', 'hogar_adul'),
                 ('education_fraction', 'escolari', 'age')
                 #('', '', ''),
                ]
    
    
  
    feats_sub = [('people_not_living', 'tamhog', 'tamviv'),
                 ('non_bedrooms', 'rooms', 'bedrooms'),
                 ('people_not_linving_', 'tamhog', 'r4t3')
                 ]

    for f_new, f1, f2 in feats_div:
        df['fe_' + f_new] = (df[f1] / df[f2]).astype(np.float32)       
    for f_new, f1, f2 in feats_sub:
        df['fe_' + f_new] = (df[f1] - df[f2]).astype(np.float32)
    
    

In [157]:
#uses do_features

def preprocess(df): #reencode idhogar
    # encode the idhogar - houshold level identifier
    df['idhogar'] = LabelEncoder().fit_transform(df['idhogar'])
    # create aggregate features
    

  

### data preprocessing

In [158]:
#mapping yes/no to float values
def yes_no_issue_solver(df):
    df['dependency'] = df['dependency'].replace({"yes": 1, "no": 0}).astype(np.float64)
    df['edjefe'] = df['edjefe'].replace({"yes": 1, "no": 0}).astype(np.float64)
    df['edjefa'] = df['edjefa'].replace({"yes": 1, "no": 0}).astype(np.float64)
    


In [336]:
def recoding_ordinal(df):  #create ordinal variable instead of OHE   
    df['walls'] = np.argmax(np.array(df[['epared1', 'epared2', 'epared3']]), axis = 1) #Get quality indexes
    df['roof'] = np.argmax(np.array(df[['etecho1', 'etecho2', 'etecho3']]), axis = 1)
    df['floor'] = np.argmax(np.array(df[['eviv1', 'eviv2', 'eviv3']]), axis = 1)
    
    power =[]
    for i, row in df.iterrows():
        if row['noelec'] == 1:
            power.append(0)
        elif row['coopele'] == 1:
            power.append(1)
        elif row['public'] == 1:
            power.append(2)
        elif row['planpri'] == 1:
            power.append(3)
        else:
            power.append(np.nan)
            
    df['power'] = power
    df['power-missing'] = df['power'].isnull()  
            
#column to drop : 'noelec', 'coopele', 'public', 'planpri', 'epared1', 'epared2', 'epared3', 'etecho1', 'etecho2', 'etecho3', 'eviv1', 'eviv2', 'eviv3'            
# data = data.drop(columns = ['etecho1', 'etecho2', 'etecho3'])

In [105]:
## Feature engineering - Aproach 1 - Handmade features

In [118]:
#list(train)

In [361]:
def agg_function_household(df): #create a lot of features
    
    list_aggr_mean = ['rez_esc', 'dis', 'male', 'female', 'estadocivil1', 'estadocivil2', 'estadocivil3', 'estadocivil4', 'estadocivil5', 'estadocivil6', 'estadocivil7', 'parentesco2',
             'parentesco3', 'parentesco4', 'parentesco5', 'parentesco6', 'parentesco7', 'parentesco8', 'parentesco9', 'parentesco10', 'parentesco11', 'parentesco12',
             'instlevel1', 'instlevel2', 'instlevel3', 'instlevel4', 'instlevel5', 'instlevel6', 'instlevel7', 'instlevel8', 'instlevel9',]

    list_other = ['escolari', 'age', 'fe_education_fraction']

    df_2 = pd.DataFrame()
    
    for item in list_aggr_mean:
        group_mean = df[item].groupby(df['idhogar']).mean()
        new_col = item + '_aggr_MEAN'
        df_2[new_col] = group_mean
        

    for item in list_other:
        for function in ['mean','std','min','max','sum']:
            group = df[item].groupby(df['idhogar']).agg(function)
            new_col = item + '_' + function.upper()
            df_2[new_col] = group
            
            
    df_2 = df_2.reset_index()
    df = pd.merge(df, df_2, on = 'idhogar')
    df.fillna(value=0, inplace=True) #check why we're doing this !
    return df
   


We still need geog averages

We need data cleaning - missing values mgmt
https://www.kaggle.com/skooch/xgboost/comments

In [362]:

train = pd.read_csv("../Costa_rica_poverty_prediction/costa-rican-household-poverty-prediction/train.csv", nrows = None)
test = pd.read_csv("../Costa_rica_poverty_prediction/costa-rican-household-poverty-prediction/test.csv")


In [339]:
def clean_column(df):
    df.drop(['idhogar','Id', 'tamhog', 'agesq', 'hogar_adul', 'SQBescolari', 'SQBage', 'SQBhogar_total', 'SQBedjefe', 'SQBhogar_nin', 'SQBovercrowding', 'SQBdependency', 'SQBmeaned', 'hhsize', 'female', 'area2'],axis=1, inplace=True)
    
    

In [358]:
for elem in [train, test]:
    feature_creation_function(elem)
    preprocess(elem) #reencode idhogar
    yes_no_issue_solver(elem)
    recoding_ordinal(elem)
    elem = agg_function_household(elem)
    clean_column(elem)

In [363]:
#incomprehensible 
feature_creation_function(train)
preprocess(train) #reencode idhogar
yes_no_issue_solver(train)
recoding_ordinal(train)
train = agg_function_household(train)
clean_column(train)

In [368]:
#list(train)

In [370]:
train.corr()['Target'].map(lambda x: abs(x)).sort_values(ascending = False)

In [372]:
X = train.query('parentesco1==1')

# pull out the target variable
y = X['Target'] - 1 # this is done to bing input labels [1,2,3,4] in agreement with lightgbm [0,1,2,3]
X = X.drop(['Target'], axis=1)

Feature selection

In [378]:
len(list(X)) #200 features

200